## Function Call 개념 이해 실습 (Tool Use)
- Function Call: LLM API가 대화 뿐만 아니라 함수도 실행하는 것
  - 단, 함수 실행의 필요성을 알려주면 실제 함수 실행은 클라이언트 또는 우리의 코드로 수행한다.

### Anthropic Claude API 사용법 복습

In [1]:
# 환경변수에 ANTHROPIC_API_KEY 추가 필요
# ~/.zshrc에 export ANTHROPIC_API_KEY='<API KEY>'
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "Hi!"}
    ]
)

print(message.content[0].text)

Greetings, young one. How may I assist you today?


### 호출 할 함수 정의
- 가장 간단한 예시로 곱셈 함수 사용

In [2]:
def multiply(a, b):
    return a * b

multiply(2, 3)

6

### 곱셈 함수를 포함한 LLM API 호출
- 질문이 곱셈 함수가 필요하다는 점을 LLM이 알려준다.
  - 곱셈과 관련이 없는 경우 ToolUseBlock이 없음

In [3]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    tools=[
       {
        "name": "multiply",
        "description": "Multiply two integers",
        "input_schema": {
            "type": "object",
            "properties": {
            "a": {
                "type": "string",
                "description": "First number"
            },
            "b": {
                "type": "string",
                "description": "Second number"
            }
            },
            "required": ["a", "b"]
        }
    }
    ],
    messages=[{"role": "user", "content": "What is the result of 2 times 3?"}],
)
response.content

[ToolUseBlock(id='toolu_01EBwhzn3baA8ch8N1BSGyjQ', input={'a': '2', 'b': '3'}, name='multiply', type='tool_use')]

In [4]:
tool_use_block = next((block for block in response.content if block.type == 'tool_use' and block.name == 'multiply'), None)
tool_use_block

ToolUseBlock(id='toolu_01EBwhzn3baA8ch8N1BSGyjQ', input={'a': '2', 'b': '3'}, name='multiply', type='tool_use')

In [5]:
if tool_use_block: 
    a = int(tool_use_block.input['a'])
    b = int(tool_use_block.input['b'])
    
    result = multiply(a, b)
    
    print(f"The result of {a} times {b} is: {result}")
else:
    print("No valid multiply tool use found in the output")

The result of 2 times 3 is: 6


### Anthropic의 공식 Function Call 예시
- 호출 할 함수 생성
- 호출 가능한 함수를 tools로 정의

In [7]:
import re

def calculate(expression):
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    }
]

In [8]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])

def chat_with_claude(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

In [9]:
chat_with_claude("What is the result of 1,984,135 * 9,343,116?")
chat_with_claude("Calculate (12851 - 593) * 301 + 76")
chat_with_claude("What is 15910385 divided by 193053?")


User Message: What is the result of 1,984,135 * 9,343,116?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text="To calculate the result of 1,984,135 * 9,343,116, we can use the calculator function available to us. Let's use it to perform this multiplication.", type='text'), ToolUseBlock(id='toolu_01HgpHFnDHkkqFhBjfTfELQ3', input={'expression': '1984135 * 9343116'}, name='calculator', type='tool_use')]

Tool Used: calculator
Tool Input: {'expression': '1984135 * 9343116'}
Tool Result: 18538003464660
[TextBlock(text='The result of the multiplication 1,984,135 * 9,343,116 is 18,538,003,464,660.\n\nTo break it down:\n- The first number is 1,984,135 (1 million, 984 thousand, 135)\n- The second number is 9,343,116 (9 million, 343 thousand, 116)\n- The product of these two numbers is 18,538,003,464,660 (18 trillion, 538 billion, 3 million, 464 thousand, 660)\n\nThis is a very large number, as expected when multiplying two numbers that are each in the millions. The calculator fu

"The result of dividing 15910385 by 193053 is approximately 82.41459599177428.\n\nTo break it down:\n- The dividend (number being divided) is 15910385\n- The divisor (number we're dividing by) is 193053\n- The quotient (result of the division) is about 82.4146\n\nThis means that 193053 goes into 15910385 about 82.4146 times. If we were to round this to two decimal places, we could say the result is 82.41.\n\nIs there anything else you'd like to know about this calculation or any other mathematical operations you'd like to perform?"

### Function Calling Lifecycle

![Function Calling](./res/function-calling-diagram.png)

*출처: OpenAI (https://platform.openai.com/docs/guides/function-calling)